In [13]:
import cv2
import pandas as pd
import re
import numpy as np
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, precision_score, recall_score, fbeta_score
from keras.models import Sequential
from keras.layers import Conv2D, LeakyReLU, MaxPooling2D, Flatten, Dense

In [15]:
TIMEFRAMES = [14, 30, 90, 180, 365]
PREDICTIONS = [5, 30, 90]
IMG_TYPES = ['OHLC', 'ColoredOHLC', 'Line', 'AlgoTrading']

labels = pd.read_csv('labels.csv')

In [4]:
# Find bounding boxes of all 4 image sizes
def find_bounding_box(img_path):
    # Load the image
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find all non-black pixels
    coords = cv2.findNonZero(gray)
    
    # Find the bounding box of those pixels
    x, y, w, h = cv2.boundingRect(coords)
    print(x, y, w, h)

# Define the paths to your images
image_paths = [
    'images/Line/AI&Robotics_2019-01-15 00:00:00_14.png',
    'images/Line/AI&Robotics_2019-01-24 00:00:00_30.png',
    'images/Line/AI&Robotics_2019-03-28 00:00:00_90.png',
    'images/Line/AI&Robotics_2019-06-20 00:00:00_180.png', 
    'images/Line/AI&Robotics_2019-12-26 00:00:00_365.png'   
]

# Crop each image and save the result
for path in image_paths:
    cropped_image = find_bounding_box(path)

80 100 85 120
80 100 132 120
80 100 226 120
80 100 414 120
80 100 602 120


In [5]:
# Crop the images using the bounding boxes
def crop_image(img_path):
    # Load the image in grayscale
    img = cv2.imread(img_path, 0)

    # Check if the image was loaded correctly
    if img is None:
        raise ValueError(f"Image at {img_path} not found. Please check the path.")

    # Use regular expression to match numbers followed by ".png" at the end of the filename
    match = re.search(r'(\d+)(?=\.png$)', img_path)
    
    # Check if we found a match
    if match:
        # Extract the number from the matched group
        number = int(match.group(1))
        
        # Check if the number is one of the specified values
        if number == 14:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+85]
        elif number == 30:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+132]
        elif number == 90:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+226]
        elif number == 180:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+414]
        elif number == 365:
            # Crop the image using the bounding rectangle
            crop = img[100:100+120, 80:80+602]
    return crop

# Example usage:
filenames = labels['Image'].values.tolist()

# Testing the function with the provided list of filenames
for name in filenames:
    try:
        cropped_image = crop_image(name)
        # Construct the new path for the cropped image
        new_path = name.replace('.png', '_cropped.png')
        # Save the cropped image
        cv2.imwrite(new_path, cropped_image)
    except ValueError as e:
        print(e)


In [16]:
# Create a new column called 'Image' that contains the path to the cropped image but only if they 
labels['Image'] = labels['Image'].str.replace('.png', '_cropped.png')

/var/folders/jx/r9zzxsjd7wxgmmn32r0l8cz40000gn/T/ipykernel_7993/666818149.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  labels['Image'] = labels['Image'].str.replace('.png', '_cropped.png')


In [17]:
# Function to load and convert an image to grayscale
def load_image(image_path):
    # Load image in grayscale
    image = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError(f"Unable to load image at path: {image_path}")
    return image

filenames = labels['Image'].values.tolist()

images = []

for name in filenames:
    try:
        img = load_image(name)
        images.append(img)
    except ValueError as e:
        print(e)

# Add a new column to the labels DataFrame to store the image arrays
labels['Image_Array'] = images

In [18]:
# Divide data based on timeframes
labels_dict = {
    "14" : labels[labels['Image'].str.contains('_14_')],
    "30" : labels[labels['Image'].str.contains('_30_')],
    "90" : labels[labels['Image'].str.contains('_90_')],
    "180" : labels[labels['Image'].str.contains('_180_')],
    "365" : labels[labels['Image'].str.contains('_365_')]
    }

In [19]:
for key in labels_dict:
    # Convert the Image_Array column to a numpy array
    labels_dict[key]['Image_Array'] = np.array(labels_dict[key]['Image_Array']) / 255.0
    # Shuffle the DataFrame
    #labels_dict[key] = shuffle(labels_dict[key], random_state=42).reset_index(drop=True)


/var/folders/jx/r9zzxsjd7wxgmmn32r0l8cz40000gn/T/ipykernel_7993/2776856819.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels_dict[key]['Image_Array'] = np.array(labels_dict[key]['Image_Array']) / 255.0


In [20]:
def create_cnn_model(num_blocks, initial_filters=32, input_shape=()):
    model = Sequential()

    for i in range(num_blocks):
        # Adjust the number of filters
        filters = initial_filters * (2 ** i)

        # Set dilation rate and strides
        dilation_rate = (2, 1) if i == 0 else (1, 1)
        strides = (1, 1)

        # Add Convolutional layer
        model.add(Conv2D(filters, kernel_size=(5, 3), strides=strides, padding='same',
                         input_shape=input_shape if i == 0 else model.output_shape[1:],
                         dilation_rate=dilation_rate))

        # Add LeakyReLU layer
        model.add(LeakyReLU(alpha=0.01))

        # Add MaxPooling layer only if dimensions allow
        if model.output_shape[1] >= 2 and model.output_shape[2] >= 2:
            model.add(MaxPooling2D(pool_size=(2, 2)))

    # Add Flatten and Dense layers for final prediction
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))  # Example Dense layer, adjust as needed
    model.add(Dense(1, activation='sigmoid'))  # Adjust the final layer as per your classification/regression needs

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # Adjust these parameters as needed

    return model

# Create models with different numbers of blocks
model_14 = create_cnn_model(1, input_shape=(120, 85, 1))
model_30 = create_cnn_model(3, input_shape=(120, 132, 1)) 
model_90 = create_cnn_model(4, input_shape=(120, 226, 1))
model_180 = create_cnn_model(5, input_shape=(120, 414, 1))
model_365 = create_cnn_model(6, input_shape=(120, 602, 1))

# Example: Print summary of one of the models
model_365.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_32 (Conv2D)          (None, 120, 602, 32)      512       
                                                                 
 leaky_re_lu_32 (LeakyReLU)  (None, 120, 602, 32)      0         
                                                                 
 max_pooling2d_32 (MaxPooli  (None, 60, 301, 32)       0         
 ng2D)                                                           
                                                                 
 conv2d_33 (Conv2D)          (None, 60, 301, 64)       30784     
                                                                 
 leaky_re_lu_33 (LeakyReLU)  (None, 60, 301, 64)       0         
                                                                 
 max_pooling2d_33 (MaxPooli  (None, 30, 150, 64)       0         
 ng2D)                                                

In [11]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical


# Split the data
X = np.array(labels_dict["14"]['Image_Array'].tolist())  # Assuming 'Image_Array' column contains image data
y = labels_dict["14"]['Label']  # Assuming 'Label' column contains labels

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

In [12]:
# Train the model
model_14.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val))

# Evaluate the model
model_14.evaluate(X_val, y_val)

Epoch 1/10
25/25 [==============================] - 5s 148ms/step - loss: 0.5426 - accuracy: 0.7465 - val_loss: 0.5722 - val_accuracy: 0.7316
Epoch 2/10
25/25 [==============================] - 3s 131ms/step - loss: 0.2967 - accuracy: 0.8758 - val_loss: 0.4282 - val_accuracy: 0.8112
Epoch 3/10
25/25 [==============================] - 3s 134ms/step - loss: 0.1835 - accuracy: 0.9265 - val_loss: 0.5224 - val_accuracy: 0.7876
Epoch 4/10
25/25 [==============================] - 3s 132ms/step - loss: 0.1014 - accuracy: 0.9785 - val_loss: 0.4623 - val_accuracy: 0.8260
Epoch 5/10
25/25 [==============================] - 3s 135ms/step - loss: 0.0511 - accuracy: 0.9886 - val_loss: 0.5005 - val_accuracy: 0.8083
Epoch 6/10
25/25 [==============================] - 3s 133ms/step - loss: 0.0331 - accuracy: 0.9962 - val_loss: 0.5737 - val_accuracy: 0.8112
Epoch 7/10
25/25 [==============================] - 3s 137ms/step - loss: 0.0156 - accuracy: 1.0000 - val_loss: 0.7499 - val_accuracy: 0.8142
Epoch 

[0.8749881386756897, 0.8112094402313232]

In [37]:
# Split the data
X = np.array(labels_dict["365"]['Image_Array'].tolist())  # Assuming 'Image_Array' column contains image data
y = labels_dict["365"]['Label']  # Assuming 'Label' column contains labels

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

In [38]:
# Train the model
model_365.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val))

# Evaluate the model
model_365.evaluate(X_val, y_val)

Epoch 1/10
23/23 [==============================] - 101s 4s/step - loss: 0.6928 - accuracy: 0.5980 - val_loss: 0.6910 - val_accuracy: 0.5359
Epoch 2/10
23/23 [==============================] - 97s 4s/step - loss: 0.6778 - accuracy: 0.5994 - val_loss: 0.6951 - val_accuracy: 0.5359
Epoch 3/10
23/23 [==============================] - 97s 4s/step - loss: 0.6745 - accuracy: 0.5994 - val_loss: 0.6944 - val_accuracy: 0.5359
Epoch 4/10
23/23 [==============================] - 98s 4s/step - loss: 0.6749 - accuracy: 0.5994 - val_loss: 0.6957 - val_accuracy: 0.5359
Epoch 5/10
23/23 [==============================] - 95s 4s/step - loss: 0.6743 - accuracy: 0.5994 - val_loss: 0.6973 - val_accuracy: 0.5359
Epoch 6/10
23/23 [==============================] - 95s 4s/step - loss: 0.6753 - accuracy: 0.5994 - val_loss: 0.6936 - val_accuracy: 0.5359
Epoch 7/10
23/23 [==============================] - 97s 4s/step - loss: 0.6758 - accuracy: 0.5994 - val_loss: 0.7099 - val_accuracy: 0.5359
Epoch 8/10
23/23 [=

[0.6999983191490173, 0.5359477400779724]

In [39]:
# Split the data
X = np.array(labels_dict["180"]['Image_Array'].tolist())  # Assuming 'Image_Array' column contains image data
y = labels_dict["180"]['Label']  # Assuming 'Label' column contains labels

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

# Train the model
model_180.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val))

# Evaluate the model
model_180.evaluate(X_val, y_val)

Epoch 1/10
25/25 [==============================] - 64s 2s/step - loss: 0.7369 - accuracy: 0.5877 - val_loss: 0.6929 - val_accuracy: 0.5234
Epoch 2/10
25/25 [==============================] - 62s 2s/step - loss: 0.6733 - accuracy: 0.5977 - val_loss: 0.6930 - val_accuracy: 0.5234
Epoch 3/10
25/25 [==============================] - 59s 2s/step - loss: 0.6919 - accuracy: 0.5977 - val_loss: 0.6927 - val_accuracy: 0.5234
Epoch 4/10
25/25 [==============================] - 59s 2s/step - loss: 0.6904 - accuracy: 0.5977 - val_loss: 0.6924 - val_accuracy: 0.5234
Epoch 5/10
25/25 [==============================] - 60s 2s/step - loss: 0.6888 - accuracy: 0.5977 - val_loss: 0.6922 - val_accuracy: 0.5234
Epoch 6/10
25/25 [==============================] - 59s 2s/step - loss: 0.6873 - accuracy: 0.5977 - val_loss: 0.6921 - val_accuracy: 0.5234
Epoch 7/10
25/25 [==============================] - 60s 2s/step - loss: 0.6860 - accuracy: 0.5977 - val_loss: 0.6921 - val_accuracy: 0.5234
Epoch 8/10
25/25 [==

[0.6922556757926941, 0.5233917832374573]

In [40]:
# Split the data
X = np.array(labels_dict["90"]['Image_Array'].tolist())  # Assuming 'Image_Array' column contains image data
y = labels_dict["90"]['Label']  # Assuming 'Label' column contains labels

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

# Train the model
model_90.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val))

# Evaluate the model
model_90.evaluate(X_val, y_val)

Epoch 1/10
18/18 [==============================] - 24s 1s/step - loss: 0.6927 - accuracy: 0.5250 - val_loss: 0.6900 - val_accuracy: 0.5417
Epoch 2/10
18/18 [==============================] - 23s 1s/step - loss: 0.6924 - accuracy: 0.5393 - val_loss: 0.6902 - val_accuracy: 0.5417
Epoch 3/10
18/18 [==============================] - 22s 1s/step - loss: 0.6910 - accuracy: 0.5393 - val_loss: 0.6900 - val_accuracy: 0.5417
Epoch 4/10
18/18 [==============================] - 23s 1s/step - loss: 0.6906 - accuracy: 0.5393 - val_loss: 0.6897 - val_accuracy: 0.5417
Epoch 5/10
18/18 [==============================] - 21s 1s/step - loss: 0.6942 - accuracy: 0.5393 - val_loss: 0.6904 - val_accuracy: 0.5417
Epoch 6/10
18/18 [==============================] - 24s 1s/step - loss: 0.6915 - accuracy: 0.5393 - val_loss: 0.6904 - val_accuracy: 0.5417
Epoch 7/10
18/18 [==============================] - 26s 1s/step - loss: 0.6909 - accuracy: 0.5393 - val_loss: 0.6899 - val_accuracy: 0.5417
Epoch 8/10
18/18 [==

[0.6896867156028748, 0.5416666865348816]

In [ ]:
for type in IMG_TYPES:
    for timeframe in TIMEFRAMES:
        for prediction in PREDICTIONS:
            if prediction < timeframe:
                print(f"Predicting {prediction} days ahead using {type} images with {timeframe} days timeframe.")
                data = labels[labels['TimePrediction'] == prediction]
                data = labels[labels['Image'].str.contains(type)]
                data = labels[labels['Image'].str.contains(f'_{timeframe}_')]
                data['Image_Array'] = np.array(data['Image_Array']) / 255.0
                data = shuffle(data, random_state=42).reset_index(drop=True)

                if timeframe == 14:
                    model = create_cnn_model(1, input_shape=(120, 85, 1))
                if timeframe == 30:
                    model = create_cnn_model(3, input_shape=(120, 132, 1))
                elif timeframe == 90:
                    model = create_cnn_model(4, input_shape=(120, 226, 1))
                elif timeframe == 180:
                    model = create_cnn_model(5, input_shape=(120, 414, 1))
                elif timeframe == 365:
                    model = create_cnn_model(5, input_shape=(120, 602, 1))
                model.summary()
                
                X = np.array(data['Image_Array'].tolist())
                y = data['Label']
                X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)
                model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val))
                model.evaluate(X_val, y_val)
                model.save(f"models/{type}_{timeframe}_{prediction}.h5")
                print(f"Model saved as {type}_{timeframe}_{prediction}.h5")
                print("--------------------------------------------------")

In [23]:
# Rolling window approach
# Create an empty DataFrame to store evaluation metrics
evaluation_df = pd.DataFrame(columns=['Image_Type', 'Timeframe', 'Prediction', 'Accuracy', 'Precision', 'Recall', 'F2_Score'])

for img_type in IMG_TYPES:
    for timeframe in TIMEFRAMES:
        for prediction in PREDICTIONS:
            if prediction < timeframe:
                print(f"Predicting {prediction} days ahead using {img_type} images with {timeframe} days timeframe.")
                
                # Filter your data based on prediction, img_type, and timeframe
                data = labels[(labels['TimePrediction'] == prediction) & (labels['Image'].str.contains(img_type)) & (labels['Image'].str.contains(f'_{timeframe}_'))]
                data = data.reset_index(drop=True)  # Reset the index to maintain temporal order
                
                # Create and compile your CNN model based on timeframe
                if timeframe == 14:
                    model = create_cnn_model(2, input_shape=(120, 85, 1))
                elif timeframe == 30:
                    model = create_cnn_model(3, input_shape=(120, 132, 1))
                elif timeframe == 90:
                    model = create_cnn_model(4, input_shape=(120, 226, 1))
                elif timeframe == 180:
                    model = create_cnn_model(5, input_shape=(120, 414, 1))
                elif timeframe == 365:
                    model = create_cnn_model(5, input_shape=(120, 602, 1))
                
                X = np.array(data['Image_Array'].tolist())
                y = data['Label']
                X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)
                
                # Train the model
                model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val))
                
                # Evaluate the model
                y_pred = model.predict(X_val)
                y_pred_binary = np.argmax(y_pred, axis=1)
                accuracy = accuracy_score(y_val, y_pred_binary)
                precision = precision_score(y_val, y_pred_binary)
                recall = recall_score(y_val, y_pred_binary)
                f2_score = fbeta_score(y_val, y_pred_binary, beta=2)
                
                print("Evaluation Metrics:")
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}")
                print(f"F2 Score: {f2_score}")
                
                # Add the evaluation metrics to the DataFrame
                evaluation_df = evaluation_df.append({
                    'Image_Type': img_type,
                    'Timeframe': timeframe,
                    'Prediction': prediction,
                    'Accuracy': accuracy,
                    'Precision': precision,
                    'Recall': recall,
                    'F2_Score': f2_score
                }, ignore_index=True)
                
                # Save the model
                model.save(f"models/{img_type}_{timeframe}_{prediction}.h5")
                print(f"Model saved as {img_type}_{timeframe}_{prediction}.h5")
                print("--------------------------------------------------")

# Save the evaluation DataFrame to a CSV file
evaluation_df.to_csv('evaluation_scores.csv', index=False)
print("Evaluation scores saved to 'evaluation_scores.csv'.")

Predicting 5 days ahead using OHLC images with 14 days timeframe.
Epoch 1/10
20/20 [==============================] - 6s 248ms/step - loss: 26.0021 - accuracy: 0.5620 - val_loss: 0.6930 - val_accuracy: 0.5506
Epoch 2/10
20/20 [==============================] - 4s 217ms/step - loss: 0.7082 - accuracy: 0.6457 - val_loss: 0.6833 - val_accuracy: 0.5506
Epoch 3/10
20/20 [==============================] - 4s 218ms/step - loss: 0.5514 - accuracy: 0.7375 - val_loss: 0.4712 - val_accuracy: 0.7790
Epoch 4/10
20/20 [==============================] - 5s 234ms/step - loss: 0.3262 - accuracy: 0.8615 - val_loss: 0.3914 - val_accuracy: 0.8390
Epoch 5/10
20/20 [==============================] - 5s 230ms/step - loss: 0.1735 - accuracy: 0.9404 - val_loss: 0.4875 - val_accuracy: 0.8052
Epoch 6/10
20/20 [==============================] - 5s 227ms/step - loss: 0.1006 - accuracy: 0.9694 - val_loss: 0.5254 - val_accuracy: 0.8090
Epoch 7/10
20/20 [==============================] - 5s 229ms/step - loss: 0.0592 

/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/10
11/11 [==============================] - 7s 519ms/step - loss: 39.1505 - accuracy: 0.4653 - val_loss: 0.6346 - val_accuracy: 0.6783
Epoch 2/10
11/11 [==============================] - 6s 524ms/step - loss: 0.5217 - accuracy: 0.7734 - val_loss: 0.6592 - val_accuracy: 0.6643
Epoch 3/10
11/11 [==============================] - 6s 512ms/step - loss: 0.3990 - accuracy: 0.8157 - val_loss: 0.6656 - val_accuracy: 0.6713
Epoch 4/10
11/11 [==============================] - 6s 518ms/step - loss: 0.2714 - accuracy: 0.8882 - val_loss: 0.4287 - val_accuracy: 0.8252
Epoch 5/10
11/11 [==============================] - 5s 468ms/step - loss: 0.1378 - accuracy: 0.9486 - val_loss: 0.4011 - val_accuracy: 0.8042
Epoch 6/10
11/11 [==============================] - 6s 544ms/step - loss: 0.0939 - accuracy: 0.9698 - val_loss: 0.3677 - val_accuracy: 0.8112
Epoch 7/10
11/11 [==============================] - 5s 467ms/step - loss: 0.0723 - accuracy: 0.9758 - val_loss: 0.3593 - val_accuracy: 0.8182
Epoch

/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/10
5/5 [==============================] - 7s 1s/step - loss: 33.1045 - accuracy: 0.5094 - val_loss: 0.7210 - val_accuracy: 0.6522
Epoch 2/10
5/5 [==============================] - 5s 1s/step - loss: 0.5639 - accuracy: 0.6792 - val_loss: 0.6838 - val_accuracy: 0.6377
Epoch 3/10
5/5 [==============================] - 6s 1s/step - loss: 0.6129 - accuracy: 0.6164 - val_loss: 0.6249 - val_accuracy: 0.6377
Epoch 4/10
5/5 [==============================] - 6s 1s/step - loss: 0.4745 - accuracy: 0.7484 - val_loss: 0.7151 - val_accuracy: 0.7246
Epoch 5/10
5/5 [==============================] - 6s 1s/step - loss: 0.4033 - accuracy: 0.8239 - val_loss: 0.7210 - val_accuracy: 0.6522
Epoch 6/10
5/5 [==============================] - 5s 1s/step - loss: 0.3710 - accuracy: 0.8176 - val_loss: 0.5514 - val_accuracy: 0.6667
Epoch 7/10
5/5 [==============================] - 5s 1s/step - loss: 0.3097 - accuracy: 0.8616 - val_loss: 0.5153 - val_accuracy: 0.7971
Epoch 8/10
5/5 [========================

...

/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/10
3/3 [==============================] - 8s 2s/step - loss: 90.7620 - accuracy: 0.4478 - val_loss: 1.4131 - val_accuracy: 0.3448
Epoch 2/10
3/3 [==============================] - 7s 2s/step - loss: 1.2958 - accuracy: 0.2836 - val_loss: 0.6478 - val_accuracy: 0.6552
Epoch 3/10
3/3 [==============================] - 7s 2s/step - loss: 0.5729 - accuracy: 0.7164 - val_loss: 0.7499 - val_accuracy: 0.6552
Epoch 4/10
3/3 [==============================] - 7s 2s/step - loss: 0.6190 - accuracy: 0.7164 - val_loss: 0.8197 - val_accuracy: 0.6552
Epoch 5/10
3/3 [==============================] - 7s 2s/step - loss: 0.6283 - accuracy: 0.7164 - val_loss: 0.6747 - val_accuracy: 0.6552
Epoch 6/10
3/3 [==============================] - 7s 2s/step - loss: 0.5531 - accuracy: 0.7164 - val_loss: 0.6404 - val_accuracy: 0.6552
Epoch 7/10
3/3 [==============================] - 7s 2s/step - loss: 0.5312 - accuracy: 0.7164 - val_loss: 0.7617 - val_accuracy: 0.6552
Epoch 8/10
3/3 [========================

/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/10
3/3 [==============================] - 8s 2s/step - loss: 143.6116 - accuracy: 0.4627 - val_loss: 2.9014 - val_accuracy: 0.2414
Epoch 2/10
3/3 [==============================] - 7s 2s/step - loss: 1.5790 - accuracy: 0.4478 - val_loss: 2.6553 - val_accuracy: 0.7586
Epoch 3/10
3/3 [==============================] - 8s 2s/step - loss: 2.2762 - accuracy: 0.5970 - val_loss: 0.6527 - val_accuracy: 0.7241
Epoch 4/10
3/3 [==============================] - 8s 2s/step - loss: 0.6536 - accuracy: 0.5672 - val_loss: 0.5106 - val_accuracy: 0.7586
Epoch 5/10
3/3 [==============================] - 7s 2s/step - loss: 0.6757 - accuracy: 0.5970 - val_loss: 0.6866 - val_accuracy: 0.7586
Epoch 6/10
3/3 [==============================] - 7s 2s/step - loss: 0.8581 - accuracy: 0.5970 - val_loss: 0.4658 - val_accuracy: 0.8966
Epoch 7/10
3/3 [==============================] - 7s 2s/step - loss: 0.5164 - accuracy: 0.7612 - val_loss: 0.4640 - val_accuracy: 0.7931
Epoch 8/10
3/3 [=======================

/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/constantinernstberger/opt/anaconda3/envs/aiss/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
